In [1]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Activation, MaxPooling2D, Input
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras import metrics, optimizers
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.callbacks import ModelCheckpoint, TensorBoard

import os
import itertools
import matplotlib.pyplot as plt
import zipfile
from time import time
import pandas as pd
import random
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix

import librosa.display
import librosa

Using TensorFlow backend.
C:\Users\Aparna\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Aparna\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Aparna\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Aparna\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [7]:
featuresdf=pd.read_pickle('featuresdf_5.pkl')  #contains mfcc features of all samples and corresponding class

In [8]:
featuresdf.head()

,feature,class_label
0,"[[-629.0347579657414, -629.0347579657414, -629...",dog
1,"[[-165.33815931479825, -173.28544807355715, -1...",chirping_birds
2,"[[-446.0381550382831, -444.05035045796683, -44...",thunderstorm
3,"[[-462.8407001928654, -463.01179671431885, -46...",thunderstorm
4,"[[-484.0664969477341, -487.5931492300499, -349...",door_wood_knock


In [9]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [10]:
# split the dataset 
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 127)

In [15]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 216
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

In [16]:
from keras.models import Sequential
from keras import layers

num_labels = yy.shape[1]

# Construct model BOAW
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))


In [17]:
model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 40

checkpointer = ModelCheckpoint(filepath='./weights.best.basic_mlp.hdf5', verbose=1, save_best_only=True)
# start = datetime.now()

history = model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)


Train on 1832 samples, validate on 459 samples
Epoch 1/100
1832/1832 [==============================] - 9s 5ms/step - loss: 4.4464 - accuracy: 0.1376 - val_loss: 2.8234 - val_accuracy: 0.1830
Epoch 2/100
1832/1832 [==============================] - 8s 4ms/step - loss: 2.4525 - accuracy: 0.2598 - val_loss: 2.4125 - val_accuracy: 0.3900
Epoch 3/100
1832/1832 [==============================] - 8s 5ms/step - loss: 2.0854 - accuracy: 0.3455 - val_loss: 2.0942 - val_accuracy: 0.4227
Epoch 4/100
1832/1832 [==============================] - 8s 4ms/step - loss: 1.8715 - accuracy: 0.4099 - val_loss: 1.9185 - val_accuracy: 0.4749
Epoch 5/100
1832/1832 [==============================] - 9s 5ms/step - loss: 1.7623 - accuracy: 0.4487 - val_loss: 1.8040 - val_accuracy: 0.4924
Epoch 6/100
1832/1832 [==============================] - 8s 5ms/step - loss: 1.5426 - accuracy: 0.4951 - val_loss: 1.6858 - val_accuracy: 0.5185
Epoch 7/100
1832/1832 [==============================] - 9s 5ms/step - loss: 1.480

In [19]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9890829920768738
Testing Accuracy:  0.8714597225189209


In [22]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [24]:
def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [28]:
def extract_features(file_name):
  audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
  mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
  mfccs = mfccs[:40,:216]
  #mfccs_processed = np.mean(mfccs.T,axis=0)
     
  return mfccs

In [33]:
import pyaudio
import wave
chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt32  # 16 bits per sample
channels = 1
fs = 44100  # Record at 44100 samples per second
seconds = 5
filename = "/Users/Aparna/Desktop/output.wav"
for i in range(1,2):
    p = pyaudio.PyAudio()  # Create an interface to PortAudio
    print('Recording')
    stream = p.open(format=sample_format, channels=channels, rate=fs, frames_per_buffer=chunk, input=True)
    frames = []  # Initialize array to store frames
    # Store data in chunks for 5 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)
    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()
    print('Finished recording')
    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()
    print_prediction(filename)
    # predictSound(filename)

Recording
Finished recording
The predicted class is: coughing 

airplane 		 :  0.00000000000868114087559934688443
breathing 		 :  0.00000001328919019272234436357394
car_horn 		 :  0.00000939905839913990348577499390
cat 		 :  0.00756435375660657882690429687500
chainsaw 		 :  0.00000232847673942160326987504959
chirping_birds 		 :  0.00000000094574559295068638675730
church_bells 		 :  0.00000005796879065655957674607635
clapping 		 :  0.00046498881420120596885681152344
clock_alarm 		 :  0.00000000009123027333579614150949
coughing 		 :  0.98489040136337280273437500000000
cow 		 :  0.00002293525903951376676559448242
crow 		 :  0.00006286464486038312315940856934
crying_baby 		 :  0.00060460029635578393936157226562
dog 		 :  0.00000065688846007105894386768341
door_wood_knock 		 :  0.00000000010331457217516160085324
engine 		 :  0.00000000556923263062003570667002
fireworks 		 :  0.00000000000632478401249580990395
helicopter 		 :  0.00000000000003498534402175505764
laughing 		 :  0.0005576804396